In [1]:
import pandas as pd

## Importing the Datasets

In [41]:
roadDistance = pd.read_csv('Dataset/RoadDistance.csv')
straightDistance = pd.read_csv('Dataset/StraightDistance.csv')

### Pre-processing the datasets

In [43]:
roadDistance.index = roadDistance["Unnamed: 0"]
roadDistance.drop(["Unnamed: 0"], axis=1, inplace=True)
roadDistance.index.name = ''
##########
straightDistance.index = roadDistance["Unnamed: 0"]
straightDistance.drop(["Unnamed: 0"], axis=1, inplace=True)
straightDistance.index.name = ''

### Datasets after preprocessing

In [45]:
roadDistance

,PashupatinathTemple,BoudhanathStupa,SwayambhunathStupa,Thamel,HanumanDhokaTemple,BuddhaNilkanth,IndraChowk,DakshinkaliTemple,RoyalBotanicalGardens,SetoMachindranathTemple,...,PatanDurbarSquare,Nagarkot,Chandragiri,Bhaktapur,Dhulikhel,Kirtipur,Phulchoki,Kulekhani,NamoBuddha,Kakani
,,,,,,,,,,,,,,,,,,,,,
PashupatinathTemple,0.0,2.1,7.5,4.1,5.00,8.8,4.80,22.9,18.1,4.60,...,6.5,19.60,15.7,13.4,29.40,12.1,29.0,49.4,40.5,25.90
BoudhanathStupa,2.1,0.0,9.1,5.7,6.80,7.4,6.50,24.6,19.9,6.40,...,8.3,24.30,17.6,12.9,29.20,13.7,30.8,48.8,40.6,27.10
SwayambhunathStupa,7.5,9.1,0.0,3.7,3.20,12.9,3.20,21.7,19.7,3.40,...,8.0,30.20,10.7,18.2,34.00,9.4,31.0,42.7,44.5,22.40
Thamel,4.1,5.7,3.7,0.0,1.60,9.5,1.60,21.3,18.1,1.80,...,5.9,27.40,12.5,17.0,31.20,9.3,29.1,44.0,42.3,23.50
HanumanDhokaTemple,5.0,6.8,3.2,1.6,0.00,10.8,0.29,19.3,16.7,0.40,...,5.1,26.70,10.5,15.4,31.30,7.2,28.5,42.0,41.6,24.50
BuddhaNilkanth,8.8,7.4,12.9,9.5,10.80,0.0,11.10,30.3,26.1,11.00,...,14.3,32.90,21.5,21.6,37.40,18.2,37.1,54.8,48.5,27.10
IndraChowk,4.8,6.5,3.2,1.6,0.29,11.1,0.00,20.0,16.6,0.10,...,5.0,29.40,10.7,15.4,31.00,7.4,27.6,42.1,41.4,24.50
DakshinkaliTemple,22.9,24.6,21.7,21.3,19.30,30.3,20.00,0.0,26.1,20.30,...,16.9,40.30,24.8,28.6,44.50,12.2,37.1,27.5,53.0,42.30
NationalBotanicalGardens,18.1,19.9,19.7,18.1,16.70,26.1,16.60,26.1,0.0,20.30,...,11.8,30.80,23.2,21.9,37.40,16.3,11.4,50.7,48.5,42.80


In [ ]:
straightDistance